In [2]:
from selenium import webdriver
import pandas as pd
import numpy as np

In [35]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urljoin

In [29]:
PATH = "/usr/local/bin/chromedriver"

In [37]:
URL = "https://oursggrants.gov.sg/grants/new"

def build_driver(headless=True):
    options = Options()
    if headless:
        options.add_argument("--headless=new")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

def scrape_grants():
    driver = build_driver(headless=False)  # non-headless to avoid anti-bot quirks
    driver.get(URL)
    wait = WebDriverWait(driver, 25)

    # Wait until at least one card is present
    wait.until(EC.presence_of_element_located(
        (By.CSS_SELECTOR, ".GrantCard_item__1FyqJ.grantItem")
    ))
    items = driver.find_elements(By.CSS_SELECTOR, ".GrantCard_item__1FyqJ.grantItem")

    results = []
    for card in items:
        # Icon src
        icon_src = None
        try:
            icon_src = card.find_element(
                By.CSS_SELECTOR, ".GrantCard_agencyIcon__2Evu3 img"
            ).get_attribute("src")
        except:
            pass

        # Title text: inner div inside the combined title container
        title_text = None
        try:
            title_text = card.find_element(
                By.CSS_SELECTOR, ".GrantCard_itemTitle__1vIcu.GrantCard_grantType__RaZ2H > div"
            ).get_property("innerText").strip()
        except:
            # Fallback: use container text
            try:
                title_text = card.find_element(
                    By.CSS_SELECTOR, ".GrantCard_itemTitle__1vIcu.GrantCard_grantType__RaZ2H"
                ).get_property("innerText").strip()
            except:
                pass

        # Description
        description_text = None
        try:
            description_text = card.find_element(
                By.CSS_SELECTOR, ".GrantCard_itemDescription__2szWZ"
            ).get_property("innerText").strip()
        except:
            pass

        # Detail link: the card is inside <a>…</a>; select the ancestor anchor
        detail_href = None
        try:
            anchor = card.find_element(By.XPATH, "./ancestor::a[1]")
            href_raw = anchor.get_dom_attribute("href") or anchor.get_attribute("href")
            detail_href = urljoin(URL, href_raw) if href_raw else None
        except Exception:
            # Fallback: go via the ancestor container and find its anchor
            try:
                container = card.find_element(
                    By.XPATH, "./ancestor::div[contains(@class,'GrantCard_cardContainer__')]"
                )
                anchor = container.find_element(By.XPATH, ".//a[@href]")
                href_raw = anchor.get_dom_attribute("href") or anchor.get_attribute("href")
                detail_href = urljoin(URL, href_raw) if href_raw else None
            except:
                pass

        results.append({
            "icon": icon_src,
            "title": title_text,
            "description": description_text,
            "detail_link": detail_href
        })

    driver.quit()
    return results

data = scrape_grants()
print(f"Fetched {len(data)} cards")
for row in data[:5]:
    print(row)

Fetched 60 cards
{'icon': 'https://oursggrants.gov.sg/AgencyIcon/sportsg/6268ff1eef22a80677d9.svg', 'title': 'Active Citizen Grant', 'description': 'The Active Citizen Grant encourages ground-up initiatives through innovating and organising activities in any of the following three domains: sports volunteerism, physical activity or health & wellness initiatives.', 'detail_link': 'https://oursggrants.gov.sg/grants/ssgacg/instruction'}
{'icon': 'https://oursggrants.gov.sg/AgencyIcon/nyc/nyc.png', 'title': 'Asia-Ready Exposure Programme (AEP)', 'description': 'Supports projects that provide youth with industry and cultural exposure to ASEAN member states, China and India, up to $1,000 per eligible youth.', 'detail_link': 'https://oursggrants.gov.sg/grants/nycaep/instruction'}
{'icon': 'https://oursggrants.gov.sg/AgencyIcon/nea/NEA%20logo%20edited.jpg', 'title': 'Call for Ideas Fund', 'description': 'The Call for Ideas Fund provides co-funding for projects that seek to resolve environmental